In [1]:
import glob
import pickle
import numpy as np
from multiprocessing import Pool
import requests, json
import pandas as pd
import glob
import timeit
import time
from multiprocessing import Pool
import numpy as np
def toc(start_time):
    elapsed = timeit.default_timer() - start_time
    print(elapsed)
from zipfile import ZipFile
import re
import xmlschema
from pprint import pprint
import glob
# importing element tree
import lxml.etree as etree
import pickle

path_codes='/mnt/sdb1/sandeep/5. NSF vs ERC/codes/'
path_data='/mnt/sdb1/sandeep/5. NSF vs ERC/data/'

my_file = open("all_country_codes.txt", "r")
all_country_codes = my_file.read()
# print(all_country_codes)

all_country_codes = all_country_codes.split("\n")[1:]
all_country_codes = [x.split(",") for x in all_country_codes]
all_country_codes


my_file = open("eu_codes.txt", "r")
eu_codes = my_file.read()
eu_codes = eu_codes.split("\n")
eu_codes = [x.split("-") for x in eu_codes]
a=[]
for x in eu_codes:
    b=[]
    for y in x:
        b.append(y.strip())
    a.append(b)
eu_codes = [x[0:2] for x in eu_codes]
eu_codes_only=[x[1].strip() for x in eu_codes]
eu_codes_only.append('GB')
eu_codes_only=sorted(eu_codes_only)
all_country_codes_only=[x[1].strip() for x in all_country_codes]

# print(all_country_codes)

# ERC: collecting names of collaborators

In [2]:


path_codes='/mnt/sdb1/sandeep/5. NSF vs ERC/codes/'
path_data='/mnt/sdb1/sandeep/5. NSF vs ERC/data/'

files_=glob.glob(path_data+'dict_careers_ERC'+'*')
#-----------------------------------------
dict_careers={}
for file in files_:
    with open(file, 'rb') as f:
        dict_careers[file.split('/')[-1]]=pickle.load(f)                



In [3]:
list_ERC_collab_patterns=[]

it_file=0
#-----------------------------------------        
t_ic = time.time();

for key in dict_careers.keys():     
    it_file=it_file+1
    scientists_nsf=list(dict_careers[key].keys())
    # IT_=11
    # for it_sci in range(IT_,IT_+1):
    for it_sci in range(1,len(scientists_nsf)):
        sci=scientists_nsf[it_sci]
#         print(sci.replace(',',''),'\n')

        x=json.loads(dict_careers[key][sci])
    #     print(sci,', works count=',len(x))
        if x!='NA':
            for it_work in range(len(x)):
                paper_=x[it_work]
#                 print(paper_.keys(),'\n\n')
#                 print(paper_['title'],'\n')
#                 print(paper_['authorships'])

                affils_=[y['institutions'] for y in paper_['authorships']]
    #--------------------------------------------------------------------------------------------
                author_pos_=[]
                countries_=[]
                author_names_=[]
    #--------------------------------------------------------------------------------------------
                for y in paper_['authorships']:
                    if 'display_name' in y['author'].keys():
                        author_names_.append(y['author']['display_name'])
                    else:
                        author_names_.append(-1)
    #--------------------------------------------------------------------------------------------
                for y in paper_['authorships']:
                    if 'author_position' in y.keys():
                        author_pos_.append(y['author_position'])
                    else:
                        author_pos_.append(-1)                        

    #--------------------------------------------------------------------------------------------
                for insti in affils_:
                    if (len(insti)>0) and ('country_code' in insti[0].keys()):
                        country=insti[0]['country_code'] ### I AM CONSIDERING ONLY THE FIRST AFFILIATION
                        countries_.append(country)
                    else:
                        countries_.append(-1)

                for y in paper_['authorships']:
                    if 'display_name' in y['author'].keys():
                        if y['author']['display_name']==sci:
                            insti_main=y['institutions']
                try:            
                    if len(insti_main)>0:
                        if 'country_code' in insti_main[0].keys():
                            country_main=insti_main[0]['country_code']
                        else:
                            country_main=None
                    else:
                        country_main=None   
                except:
                    country_main=None   
                try:
                    del insti_main
                except:
                    ''                    
    #--------------------------------------------------------------------------------------------

                list_ERC_collab_patterns.append({'scientist':sci.replace(',',''),
                                                 'countries':countries_,
                                                 'team_size':len(paper_['authorships']),
                                                 'pub_year':paper_['publication_year'],
                                                 'counts_by_year':paper_['counts_by_year'],
                                                 'country_awardee':country_main,
                                                 'open_access':paper_['open_access'],
                                                 'title':paper_['title'],
                                                 'concepts':paper_['concepts'],
                                                 'author_pos_':author_pos_,
                                                 'author_names_':author_names_
                                                })
                

#     print(it_file,len(dict_careers.keys()),end='\r')
    
    t_oc = time.time();
    progress='progress'+'--'+str(it_file)+ '---- '+str(it_file/len(dict_careers.keys()))+' '+str(round(t_oc-t_ic,2))+'--estimated---'+str(round((t_oc-t_ic)/(it_file/len(dict_careers.keys()))/3600,3))+'hours'

    print(progress,end='\r')
    with open(path_codes+"1_df_ERC_tracker.txt", "w") as file_object:
        file_object.write(progress+'\n')

    

In [6]:
with open(path_data+'list_ERC_collab_patterns(frequency of countries).pkl', 'wb') as f:
    pickle.dump(list_ERC_collab_patterns, f)

In [ ]:
dfs_ERC= pd.DataFrame(list_ERC_collab_patterns)

dfs_ERC=dfs_ERC.sort_values(['scientist','pub_year'],ascending=True)
countries_temp=list(dfs_ERC['countries'])

n_identified=[len(list(filter(None, x))) for x in countries_temp]
dfs_ERC['identified_countries']=n_identified
dfs_ERC['n_US']=[np.sum(np.array(x)=='US') for x in countries_temp]
dfs_ERC['n_EU']=[np.sum(np.in1d(x,eu_codes_only)) for x in countries_temp]        
for country in ['IN','CN','JP','BR']:
    dfs_ERC['n_'+country]=[np.sum(np.array(x)==country) for x in countries_temp]        
dfs_ERC=dfs_ERC.drop(columns=['author_pos_','counts_by_year', 'country_awardee','open_access','title','concepts','author_names_'])
    
with open(path_data+'dfs_ERC_china_etc.pkl', 'wb') as f:
    pickle.dump(dfs_ERC, f)


<ipython-input-4-d8bd093e0270>:8: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  dfs_ERC['n_US']=[np.sum(np.array(x)=='US') for x in countries_temp]
/mnt/sdb1/sandeep/miniconda3/envs/sos/lib/python3.9/site-packages/numpy/lib/arraysetops.py:583: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [7]:
# path_codes='/mnt/sdb1/sandeep/5. NSF vs ERC/codes/'
# path_data='/mnt/sdb1/sandeep/5. NSF vs ERC/data/'       

# with open(path_data+'dfs_ERC_with_collab_names.pkl', 'rb') as f:
#     dfs_ERC=pickle.load(f)   

In [5]:
# Query to get career staring year for each author.

